# Цель работы:
 Исследовать сходство и различия в толковании понятий "machine learning" и "data science" через анализ публикаций в Arxive с использованием метода обобщения в таксономиях

# Шаг 1
Выделить подмножества публикаций в Arxive, ML и DS, отвечающие соответствующим запросам

In [2]:
import pandas as pd
import numpy as np

df = pd.read_json('arxiv-metadata-oai-snapshot.json', lines=True)

Filter ML and AI from past 10 years

In [3]:
df_LG = df[df['categories'].str.contains('cs.LG')]

In [4]:
df_AI = df[df['categories'].str.contains('cs.AI')]

In [5]:
df_ML = df[df['categories'].str.contains('stat.ML')]

In [6]:
df_LG = df_LG[df_LG['update_date'] > '2022']

In [7]:
df_AI = df_AI[df_AI['update_date'] > '2022']

In [8]:
df_ML = df_ML[df_ML['update_date'] > '2022']

In [9]:
print(f'ML articles: {len(df_LG)}')
print(f'AI articles: {len(df_AI)}')
print(f'ML (statistics) articles: {len(df_ML)}')

ML articles: 42929
AI articles: 20841
ML (statistics) articles: 9336


In [10]:
abstracts_LG = []
for a in df_LG['abstract']:
    abstracts_LG.append(a)

In [11]:
abstracts_AI = []

for a in df_AI['abstract']:
    abstracts_AI.append(a)

In [12]:
abstracts_ML = []

for a in df_ML['abstract']:
    abstracts_ML.append(a)

## Шаг 2
Выделить разумные нечеткие кластеры понятий таксономии согласно ML и согласно DS

In [13]:
import re

import numpy as np

from east.asts import base


def clear_text(text, lowerize=True):

    pat = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
    cleared_text = re.sub(pat, ' ', text)

    if lowerize:
        cleared_text = cleared_text.lower()

    tokens = cleared_text.split()
    return tokens


def make_substrings(tokens, k=4):

    for i in range(max(len(tokens) - k + 1, 1)):
        yield ' '.join(tokens[i:i + k])


def get_corelevance_matrix(texts):

    matrix = np.empty((0, len(texts)), float)
    prepared_text_tokens = [clear_text(t) for t in texts]
    prepared_texts = [' '.join(t) for t in prepared_text_tokens]

    for text_tokens in prepared_text_tokens:
        ast = base.AST.get_ast(list(make_substrings(text_tokens)))
        row = np.array([ast.score(t) for t in prepared_texts])
        matrix = np.append(matrix, [row], axis=0)

    return matrix


def get_relevance_matrix(texts, strings):

    matrix = np.empty((0, len(strings)), float)
    prepared_text_tokens = [clear_text(t) for t in texts]

    prepared_string_tokens = [clear_text(s) for s in strings]
    prepared_strings = [' '.join(t) for t in prepared_string_tokens]
    
    #for print
    print(f'total len: {len(prepared_text_tokens)}')
    c = 0

    for text_tokens in prepared_text_tokens:
        ast = base.AST.get_ast(list(make_substrings(text_tokens)))
        row = np.array([ast.score(s) for s in prepared_strings])
        matrix = np.append(matrix, [row], axis=0)
        
        c += 1
        print(f'processed: {c}')
    return matrix


def save_matrix(matrix, filename="filename"):
    np.savetxt(filename, matrix)

Get strings of terms

In [14]:
with open("taxonomies/taxonomy_leaves.txt") as f:
    strings = [l.strip() for l in f.readlines()]

In [15]:
# Relevance between the texts and strings:

relevance_matrix_LG = get_relevance_matrix(abstracts_LG, strings)
print("Relevance mairix LG:")
print(relevance_matrix_LG)

save_matrix(relevance_matrix_LG, "matrixes/relevmtx_LG")

total len: 42929
processed: 1
processed: 2
processed: 3
processed: 4
processed: 5
processed: 6
processed: 7
processed: 8
processed: 9
processed: 10
processed: 11
processed: 12
processed: 13
processed: 14
processed: 15
processed: 16
processed: 17
processed: 18
processed: 19
processed: 20
processed: 21
processed: 22
processed: 23
processed: 24
processed: 25
processed: 26
processed: 27
processed: 28
processed: 29
processed: 30
processed: 31
processed: 32
processed: 33
processed: 34
processed: 35
processed: 36
processed: 37
processed: 38
processed: 39
processed: 40
processed: 41
processed: 42
processed: 43
processed: 44
processed: 45
processed: 46
processed: 47
processed: 48
processed: 49
processed: 50
processed: 51
processed: 52
processed: 53
processed: 54
processed: 55
processed: 56
processed: 57
processed: 58
processed: 59
processed: 60
processed: 61
processed: 62
processed: 63
processed: 64
processed: 65
processed: 66
processed: 67
processed: 68
processed: 69
processed: 70
processed: 7

processed: 554
processed: 555
processed: 556
processed: 557
processed: 558
processed: 559
processed: 560
processed: 561
processed: 562
processed: 563
processed: 564
processed: 565
processed: 566
processed: 567
processed: 568
processed: 569
processed: 570
processed: 571
processed: 572
processed: 573
processed: 574
processed: 575
processed: 576
processed: 577
processed: 578
processed: 579
processed: 580
processed: 581
processed: 582
processed: 583
processed: 584
processed: 585
processed: 586
processed: 587
processed: 588
processed: 589
processed: 590
processed: 591
processed: 592
processed: 593
processed: 594
processed: 595
processed: 596
processed: 597
processed: 598
processed: 599
processed: 600
processed: 601
processed: 602
processed: 603
processed: 604
processed: 605
processed: 606
processed: 607
processed: 608
processed: 609
processed: 610
processed: 611
processed: 612
processed: 613
processed: 614
processed: 615
processed: 616
processed: 617
processed: 618
processed: 619
processed:

processed: 1094
processed: 1095
processed: 1096
processed: 1097
processed: 1098
processed: 1099
processed: 1100
processed: 1101
processed: 1102
processed: 1103
processed: 1104
processed: 1105
processed: 1106
processed: 1107
processed: 1108
processed: 1109
processed: 1110
processed: 1111
processed: 1112
processed: 1113
processed: 1114
processed: 1115
processed: 1116
processed: 1117
processed: 1118
processed: 1119
processed: 1120
processed: 1121
processed: 1122
processed: 1123
processed: 1124
processed: 1125
processed: 1126
processed: 1127
processed: 1128
processed: 1129
processed: 1130
processed: 1131
processed: 1132
processed: 1133
processed: 1134
processed: 1135
processed: 1136
processed: 1137
processed: 1138
processed: 1139
processed: 1140
processed: 1141
processed: 1142
processed: 1143
processed: 1144
processed: 1145
processed: 1146
processed: 1147
processed: 1148
processed: 1149
processed: 1150
processed: 1151
processed: 1152
processed: 1153
processed: 1154
processed: 1155
processe

processed: 1607
processed: 1608
processed: 1609
processed: 1610
processed: 1611
processed: 1612
processed: 1613
processed: 1614
processed: 1615
processed: 1616
processed: 1617
processed: 1618
processed: 1619
processed: 1620
processed: 1621
processed: 1622
processed: 1623
processed: 1624
processed: 1625
processed: 1626
processed: 1627
processed: 1628
processed: 1629
processed: 1630
processed: 1631
processed: 1632
processed: 1633
processed: 1634
processed: 1635
processed: 1636
processed: 1637
processed: 1638
processed: 1639
processed: 1640
processed: 1641
processed: 1642
processed: 1643
processed: 1644
processed: 1645
processed: 1646
processed: 1647
processed: 1648
processed: 1649
processed: 1650
processed: 1651
processed: 1652
processed: 1653
processed: 1654
processed: 1655
processed: 1656
processed: 1657
processed: 1658
processed: 1659
processed: 1660
processed: 1661
processed: 1662
processed: 1663
processed: 1664
processed: 1665
processed: 1666
processed: 1667
processed: 1668
processe

processed: 2120
processed: 2121
processed: 2122
processed: 2123
processed: 2124
processed: 2125
processed: 2126
processed: 2127
processed: 2128
processed: 2129
processed: 2130
processed: 2131
processed: 2132
processed: 2133
processed: 2134
processed: 2135
processed: 2136
processed: 2137
processed: 2138
processed: 2139
processed: 2140
processed: 2141
processed: 2142
processed: 2143
processed: 2144
processed: 2145
processed: 2146
processed: 2147
processed: 2148
processed: 2149
processed: 2150
processed: 2151
processed: 2152
processed: 2153
processed: 2154
processed: 2155
processed: 2156
processed: 2157
processed: 2158
processed: 2159
processed: 2160
processed: 2161
processed: 2162
processed: 2163
processed: 2164
processed: 2165
processed: 2166
processed: 2167
processed: 2168
processed: 2169
processed: 2170
processed: 2171
processed: 2172
processed: 2173
processed: 2174
processed: 2175
processed: 2176
processed: 2177
processed: 2178
processed: 2179
processed: 2180
processed: 2181
processe

...

processed: 40675
processed: 40676
processed: 40677
processed: 40678
processed: 40679
processed: 40680
processed: 40681
processed: 40682
processed: 40683
processed: 40684
processed: 40685
processed: 40686
processed: 40687
processed: 40688
processed: 40689
processed: 40690
processed: 40691
processed: 40692
processed: 40693
processed: 40694
processed: 40695
processed: 40696
processed: 40697
processed: 40698
processed: 40699
processed: 40700
processed: 40701
processed: 40702
processed: 40703
processed: 40704
processed: 40705
processed: 40706
processed: 40707
processed: 40708
processed: 40709
processed: 40710
processed: 40711
processed: 40712
processed: 40713
processed: 40714
processed: 40715
processed: 40716
processed: 40717
processed: 40718
processed: 40719
processed: 40720
processed: 40721
processed: 40722
processed: 40723
processed: 40724
processed: 40725
processed: 40726
processed: 40727
processed: 40728
processed: 40729
processed: 40730
processed: 40731
processed: 40732
processed: 407

processed: 41157
processed: 41158
processed: 41159
processed: 41160
processed: 41161
processed: 41162
processed: 41163
processed: 41164
processed: 41165
processed: 41166
processed: 41167
processed: 41168
processed: 41169
processed: 41170
processed: 41171
processed: 41172
processed: 41173
processed: 41174
processed: 41175
processed: 41176
processed: 41177
processed: 41178
processed: 41179
processed: 41180
processed: 41181
processed: 41182
processed: 41183
processed: 41184
processed: 41185
processed: 41186
processed: 41187
processed: 41188
processed: 41189
processed: 41190
processed: 41191
processed: 41192
processed: 41193
processed: 41194
processed: 41195
processed: 41196
processed: 41197
processed: 41198
processed: 41199
processed: 41200
processed: 41201
processed: 41202
processed: 41203
processed: 41204
processed: 41205
processed: 41206
processed: 41207
processed: 41208
processed: 41209
processed: 41210
processed: 41211
processed: 41212
processed: 41213
processed: 41214
processed: 412

processed: 41639
processed: 41640
processed: 41641
processed: 41642
processed: 41643
processed: 41644
processed: 41645
processed: 41646
processed: 41647
processed: 41648
processed: 41649
processed: 41650
processed: 41651
processed: 41652
processed: 41653
processed: 41654
processed: 41655
processed: 41656
processed: 41657
processed: 41658
processed: 41659
processed: 41660
processed: 41661
processed: 41662
processed: 41663
processed: 41664
processed: 41665
processed: 41666
processed: 41667
processed: 41668
processed: 41669
processed: 41670
processed: 41671
processed: 41672
processed: 41673
processed: 41674
processed: 41675
processed: 41676
processed: 41677
processed: 41678
processed: 41679
processed: 41680
processed: 41681
processed: 41682
processed: 41683
processed: 41684
processed: 41685
processed: 41686
processed: 41687
processed: 41688
processed: 41689
processed: 41690
processed: 41691
processed: 41692
processed: 41693
processed: 41694
processed: 41695
processed: 41696
processed: 416

processed: 42121
processed: 42122
processed: 42123
processed: 42124
processed: 42125
processed: 42126
processed: 42127
processed: 42128
processed: 42129
processed: 42130
processed: 42131
processed: 42132
processed: 42133
processed: 42134
processed: 42135
processed: 42136
processed: 42137
processed: 42138
processed: 42139
processed: 42140
processed: 42141
processed: 42142
processed: 42143
processed: 42144
processed: 42145
processed: 42146
processed: 42147
processed: 42148
processed: 42149
processed: 42150
processed: 42151
processed: 42152
processed: 42153
processed: 42154
processed: 42155
processed: 42156
processed: 42157
processed: 42158
processed: 42159
processed: 42160
processed: 42161
processed: 42162
processed: 42163
processed: 42164
processed: 42165
processed: 42166
processed: 42167
processed: 42168
processed: 42169
processed: 42170
processed: 42171
processed: 42172
processed: 42173
processed: 42174
processed: 42175
processed: 42176
processed: 42177
processed: 42178
processed: 421

processed: 42603
processed: 42604
processed: 42605
processed: 42606
processed: 42607
processed: 42608
processed: 42609
processed: 42610
processed: 42611
processed: 42612
processed: 42613
processed: 42614
processed: 42615
processed: 42616
processed: 42617
processed: 42618
processed: 42619
processed: 42620
processed: 42621
processed: 42622
processed: 42623
processed: 42624
processed: 42625
processed: 42626
processed: 42627
processed: 42628
processed: 42629
processed: 42630
processed: 42631
processed: 42632
processed: 42633
processed: 42634
processed: 42635
processed: 42636
processed: 42637
processed: 42638
processed: 42639
processed: 42640
processed: 42641
processed: 42642
processed: 42643
processed: 42644
processed: 42645
processed: 42646
processed: 42647
processed: 42648
processed: 42649
processed: 42650
processed: 42651
processed: 42652
processed: 42653
processed: 42654
processed: 42655
processed: 42656
processed: 42657
processed: 42658
processed: 42659
processed: 42660
processed: 426

In [16]:
# Relevance between the texts and strings:

relevance_matrix_AI = get_relevance_matrix(abstracts_AI, strings)
print("Relevance mairix AI:")
print(relevance_matrix_AI)

save_matrix(relevance_matrix_AI, "matrixes/relevmtx_AI")

total len: 20841
processed: 1
processed: 2
processed: 3
processed: 4
processed: 5
processed: 6
processed: 7
processed: 8
processed: 9
processed: 10
processed: 11
processed: 12
processed: 13
processed: 14
processed: 15
processed: 16
processed: 17
processed: 18
processed: 19
processed: 20
processed: 21
processed: 22
processed: 23
processed: 24
processed: 25
processed: 26
processed: 27
processed: 28
processed: 29
processed: 30
processed: 31
processed: 32
processed: 33
processed: 34
processed: 35
processed: 36
processed: 37
processed: 38
processed: 39
processed: 40
processed: 41
processed: 42
processed: 43
processed: 44
processed: 45
processed: 46
processed: 47
processed: 48
processed: 49
processed: 50
processed: 51
processed: 52
processed: 53
processed: 54
processed: 55
processed: 56
processed: 57
processed: 58
processed: 59
processed: 60
processed: 61
processed: 62
processed: 63
processed: 64
processed: 65
processed: 66
processed: 67
processed: 68
processed: 69
processed: 70
processed: 7

processed: 554
processed: 555
processed: 556
processed: 557
processed: 558
processed: 559
processed: 560
processed: 561
processed: 562
processed: 563
processed: 564
processed: 565
processed: 566
processed: 567
processed: 568
processed: 569
processed: 570
processed: 571
processed: 572
processed: 573
processed: 574
processed: 575
processed: 576
processed: 577
processed: 578
processed: 579
processed: 580
processed: 581
processed: 582
processed: 583
processed: 584
processed: 585
processed: 586
processed: 587
processed: 588
processed: 589
processed: 590
processed: 591
processed: 592
processed: 593
processed: 594
processed: 595
processed: 596
processed: 597
processed: 598
processed: 599
processed: 600
processed: 601
processed: 602
processed: 603
processed: 604
processed: 605
processed: 606
processed: 607
processed: 608
processed: 609
processed: 610
processed: 611
processed: 612
processed: 613
processed: 614
processed: 615
processed: 616
processed: 617
processed: 618
processed: 619
processed:

processed: 1094
processed: 1095
processed: 1096
processed: 1097
processed: 1098
processed: 1099
processed: 1100
processed: 1101
processed: 1102
processed: 1103
processed: 1104
processed: 1105
processed: 1106
processed: 1107
processed: 1108
processed: 1109
processed: 1110
processed: 1111
processed: 1112
processed: 1113
processed: 1114
processed: 1115
processed: 1116
processed: 1117
processed: 1118
processed: 1119
processed: 1120
processed: 1121
processed: 1122
processed: 1123
processed: 1124
processed: 1125
processed: 1126
processed: 1127
processed: 1128
processed: 1129
processed: 1130
processed: 1131
processed: 1132
processed: 1133
processed: 1134
processed: 1135
processed: 1136
processed: 1137
processed: 1138
processed: 1139
processed: 1140
processed: 1141
processed: 1142
processed: 1143
processed: 1144
processed: 1145
processed: 1146
processed: 1147
processed: 1148
processed: 1149
processed: 1150
processed: 1151
processed: 1152
processed: 1153
processed: 1154
processed: 1155
processe

processed: 1607
processed: 1608
processed: 1609
processed: 1610
processed: 1611
processed: 1612
processed: 1613
processed: 1614
processed: 1615
processed: 1616
processed: 1617
processed: 1618
processed: 1619
processed: 1620
processed: 1621
processed: 1622
processed: 1623
processed: 1624
processed: 1625
processed: 1626
processed: 1627
processed: 1628
processed: 1629
processed: 1630
processed: 1631
processed: 1632
processed: 1633
processed: 1634
processed: 1635
processed: 1636
processed: 1637
processed: 1638
processed: 1639
processed: 1640
processed: 1641
processed: 1642
processed: 1643
processed: 1644
processed: 1645
processed: 1646
processed: 1647
processed: 1648
processed: 1649
processed: 1650
processed: 1651
processed: 1652
processed: 1653
processed: 1654
processed: 1655
processed: 1656
processed: 1657
processed: 1658
processed: 1659
processed: 1660
processed: 1661
processed: 1662
processed: 1663
processed: 1664
processed: 1665
processed: 1666
processed: 1667
processed: 1668
processe

processed: 2120
processed: 2121
processed: 2122
processed: 2123
processed: 2124
processed: 2125
processed: 2126
processed: 2127
processed: 2128
processed: 2129
processed: 2130
processed: 2131
processed: 2132
processed: 2133
processed: 2134
processed: 2135
processed: 2136
processed: 2137
processed: 2138
processed: 2139
processed: 2140
processed: 2141
processed: 2142
processed: 2143
processed: 2144
processed: 2145
processed: 2146
processed: 2147
processed: 2148
processed: 2149
processed: 2150
processed: 2151
processed: 2152
processed: 2153
processed: 2154
processed: 2155
processed: 2156
processed: 2157
processed: 2158
processed: 2159
processed: 2160
processed: 2161
processed: 2162
processed: 2163
processed: 2164
processed: 2165
processed: 2166
processed: 2167
processed: 2168
processed: 2169
processed: 2170
processed: 2171
processed: 2172
processed: 2173
processed: 2174
processed: 2175
processed: 2176
processed: 2177
processed: 2178
processed: 2179
processed: 2180
processed: 2181
processe

...

processed: 18502
processed: 18503
processed: 18504
processed: 18505
processed: 18506
processed: 18507
processed: 18508
processed: 18509
processed: 18510
processed: 18511
processed: 18512
processed: 18513
processed: 18514
processed: 18515
processed: 18516
processed: 18517
processed: 18518
processed: 18519
processed: 18520
processed: 18521
processed: 18522
processed: 18523
processed: 18524
processed: 18525
processed: 18526
processed: 18527
processed: 18528
processed: 18529
processed: 18530
processed: 18531
processed: 18532
processed: 18533
processed: 18534
processed: 18535
processed: 18536
processed: 18537
processed: 18538
processed: 18539
processed: 18540
processed: 18541
processed: 18542
processed: 18543
processed: 18544
processed: 18545
processed: 18546
processed: 18547
processed: 18548
processed: 18549
processed: 18550
processed: 18551
processed: 18552
processed: 18553
processed: 18554
processed: 18555
processed: 18556
processed: 18557
processed: 18558
processed: 18559
processed: 185

processed: 18984
processed: 18985
processed: 18986
processed: 18987
processed: 18988
processed: 18989
processed: 18990
processed: 18991
processed: 18992
processed: 18993
processed: 18994
processed: 18995
processed: 18996
processed: 18997
processed: 18998
processed: 18999
processed: 19000
processed: 19001
processed: 19002
processed: 19003
processed: 19004
processed: 19005
processed: 19006
processed: 19007
processed: 19008
processed: 19009
processed: 19010
processed: 19011
processed: 19012
processed: 19013
processed: 19014
processed: 19015
processed: 19016
processed: 19017
processed: 19018
processed: 19019
processed: 19020
processed: 19021
processed: 19022
processed: 19023
processed: 19024
processed: 19025
processed: 19026
processed: 19027
processed: 19028
processed: 19029
processed: 19030
processed: 19031
processed: 19032
processed: 19033
processed: 19034
processed: 19035
processed: 19036
processed: 19037
processed: 19038
processed: 19039
processed: 19040
processed: 19041
processed: 190

processed: 19466
processed: 19467
processed: 19468
processed: 19469
processed: 19470
processed: 19471
processed: 19472
processed: 19473
processed: 19474
processed: 19475
processed: 19476
processed: 19477
processed: 19478
processed: 19479
processed: 19480
processed: 19481
processed: 19482
processed: 19483
processed: 19484
processed: 19485
processed: 19486
processed: 19487
processed: 19488
processed: 19489
processed: 19490
processed: 19491
processed: 19492
processed: 19493
processed: 19494
processed: 19495
processed: 19496
processed: 19497
processed: 19498
processed: 19499
processed: 19500
processed: 19501
processed: 19502
processed: 19503
processed: 19504
processed: 19505
processed: 19506
processed: 19507
processed: 19508
processed: 19509
processed: 19510
processed: 19511
processed: 19512
processed: 19513
processed: 19514
processed: 19515
processed: 19516
processed: 19517
processed: 19518
processed: 19519
processed: 19520
processed: 19521
processed: 19522
processed: 19523
processed: 195

processed: 19948
processed: 19949
processed: 19950
processed: 19951
processed: 19952
processed: 19953
processed: 19954
processed: 19955
processed: 19956
processed: 19957
processed: 19958
processed: 19959
processed: 19960
processed: 19961
processed: 19962
processed: 19963
processed: 19964
processed: 19965
processed: 19966
processed: 19967
processed: 19968
processed: 19969
processed: 19970
processed: 19971
processed: 19972
processed: 19973
processed: 19974
processed: 19975
processed: 19976
processed: 19977
processed: 19978
processed: 19979
processed: 19980
processed: 19981
processed: 19982
processed: 19983
processed: 19984
processed: 19985
processed: 19986
processed: 19987
processed: 19988
processed: 19989
processed: 19990
processed: 19991
processed: 19992
processed: 19993
processed: 19994
processed: 19995
processed: 19996
processed: 19997
processed: 19998
processed: 19999
processed: 20000
processed: 20001
processed: 20002
processed: 20003
processed: 20004
processed: 20005
processed: 200

processed: 20430
processed: 20431
processed: 20432
processed: 20433
processed: 20434
processed: 20435
processed: 20436
processed: 20437
processed: 20438
processed: 20439
processed: 20440
processed: 20441
processed: 20442
processed: 20443
processed: 20444
processed: 20445
processed: 20446
processed: 20447
processed: 20448
processed: 20449
processed: 20450
processed: 20451
processed: 20452
processed: 20453
processed: 20454
processed: 20455
processed: 20456
processed: 20457
processed: 20458
processed: 20459
processed: 20460
processed: 20461
processed: 20462
processed: 20463
processed: 20464
processed: 20465
processed: 20466
processed: 20467
processed: 20468
processed: 20469
processed: 20470
processed: 20471
processed: 20472
processed: 20473
processed: 20474
processed: 20475
processed: 20476
processed: 20477
processed: 20478
processed: 20479
processed: 20480
processed: 20481
processed: 20482
processed: 20483
processed: 20484
processed: 20485
processed: 20486
processed: 20487
processed: 204

In [17]:
# Relevance between the texts and strings:

relevance_matrix_ML = get_relevance_matrix(abstracts_ML, strings)
print("Relevance mairix ML:")
print(relevance_matrix_ML)

save_matrix(relevance_matrix_ML, "matrixes/relevmtx_ML")

total len: 9336
processed: 1
processed: 2
processed: 3
processed: 4
processed: 5
processed: 6
processed: 7
processed: 8
processed: 9
processed: 10
processed: 11
processed: 12
processed: 13
processed: 14
processed: 15
processed: 16
processed: 17
processed: 18
processed: 19
processed: 20
processed: 21
processed: 22
processed: 23
processed: 24
processed: 25
processed: 26
processed: 27
processed: 28
processed: 29
processed: 30
processed: 31
processed: 32
processed: 33
processed: 34
processed: 35
processed: 36
processed: 37
processed: 38
processed: 39
processed: 40
processed: 41
processed: 42
processed: 43
processed: 44
processed: 45
processed: 46
processed: 47
processed: 48
processed: 49
processed: 50
processed: 51
processed: 52
processed: 53
processed: 54
processed: 55
processed: 56
processed: 57
processed: 58
processed: 59
processed: 60
processed: 61
processed: 62
processed: 63
processed: 64
processed: 65
processed: 66
processed: 67
processed: 68
processed: 69
processed: 70
processed: 71

processed: 555
processed: 556
processed: 557
processed: 558
processed: 559
processed: 560
processed: 561
processed: 562
processed: 563
processed: 564
processed: 565
processed: 566
processed: 567
processed: 568
processed: 569
processed: 570
processed: 571
processed: 572
processed: 573
processed: 574
processed: 575
processed: 576
processed: 577
processed: 578
processed: 579
processed: 580
processed: 581
processed: 582
processed: 583
processed: 584
processed: 585
processed: 586
processed: 587
processed: 588
processed: 589
processed: 590
processed: 591
processed: 592
processed: 593
processed: 594
processed: 595
processed: 596
processed: 597
processed: 598
processed: 599
processed: 600
processed: 601
processed: 602
processed: 603
processed: 604
processed: 605
processed: 606
processed: 607
processed: 608
processed: 609
processed: 610
processed: 611
processed: 612
processed: 613
processed: 614
processed: 615
processed: 616
processed: 617
processed: 618
processed: 619
processed: 620
processed:

processed: 1095
processed: 1096
processed: 1097
processed: 1098
processed: 1099
processed: 1100
processed: 1101
processed: 1102
processed: 1103
processed: 1104
processed: 1105
processed: 1106
processed: 1107
processed: 1108
processed: 1109
processed: 1110
processed: 1111
processed: 1112
processed: 1113
processed: 1114
processed: 1115
processed: 1116
processed: 1117
processed: 1118
processed: 1119
processed: 1120
processed: 1121
processed: 1122
processed: 1123
processed: 1124
processed: 1125
processed: 1126
processed: 1127
processed: 1128
processed: 1129
processed: 1130
processed: 1131
processed: 1132
processed: 1133
processed: 1134
processed: 1135
processed: 1136
processed: 1137
processed: 1138
processed: 1139
processed: 1140
processed: 1141
processed: 1142
processed: 1143
processed: 1144
processed: 1145
processed: 1146
processed: 1147
processed: 1148
processed: 1149
processed: 1150
processed: 1151
processed: 1152
processed: 1153
processed: 1154
processed: 1155
processed: 1156
processe

processed: 1608
processed: 1609
processed: 1610
processed: 1611
processed: 1612
processed: 1613
processed: 1614
processed: 1615
processed: 1616
processed: 1617
processed: 1618
processed: 1619
processed: 1620
processed: 1621
processed: 1622
processed: 1623
processed: 1624
processed: 1625
processed: 1626
processed: 1627
processed: 1628
processed: 1629
processed: 1630
processed: 1631
processed: 1632
processed: 1633
processed: 1634
processed: 1635
processed: 1636
processed: 1637
processed: 1638
processed: 1639
processed: 1640
processed: 1641
processed: 1642
processed: 1643
processed: 1644
processed: 1645
processed: 1646
processed: 1647
processed: 1648
processed: 1649
processed: 1650
processed: 1651
processed: 1652
processed: 1653
processed: 1654
processed: 1655
processed: 1656
processed: 1657
processed: 1658
processed: 1659
processed: 1660
processed: 1661
processed: 1662
processed: 1663
processed: 1664
processed: 1665
processed: 1666
processed: 1667
processed: 1668
processed: 1669
processe

processed: 2121
processed: 2122
processed: 2123
processed: 2124
processed: 2125
processed: 2126
processed: 2127
processed: 2128
processed: 2129
processed: 2130
processed: 2131
processed: 2132
processed: 2133
processed: 2134
processed: 2135
processed: 2136
processed: 2137
processed: 2138
processed: 2139
processed: 2140
processed: 2141
processed: 2142
processed: 2143
processed: 2144
processed: 2145
processed: 2146
processed: 2147
processed: 2148
processed: 2149
processed: 2150
processed: 2151
processed: 2152
processed: 2153
processed: 2154
processed: 2155
processed: 2156
processed: 2157
processed: 2158
processed: 2159
processed: 2160
processed: 2161
processed: 2162
processed: 2163
processed: 2164
processed: 2165
processed: 2166
processed: 2167
processed: 2168
processed: 2169
processed: 2170
processed: 2171
processed: 2172
processed: 2173
processed: 2174
processed: 2175
processed: 2176
processed: 2177
processed: 2178
processed: 2179
processed: 2180
processed: 2181
processed: 2182
processe

...

processed: 7251
processed: 7252
processed: 7253
processed: 7254
processed: 7255
processed: 7256
processed: 7257
processed: 7258
processed: 7259
processed: 7260
processed: 7261
processed: 7262
processed: 7263
processed: 7264
processed: 7265
processed: 7266
processed: 7267
processed: 7268
processed: 7269
processed: 7270
processed: 7271
processed: 7272
processed: 7273
processed: 7274
processed: 7275
processed: 7276
processed: 7277
processed: 7278
processed: 7279
processed: 7280
processed: 7281
processed: 7282
processed: 7283
processed: 7284
processed: 7285
processed: 7286
processed: 7287
processed: 7288
processed: 7289
processed: 7290
processed: 7291
processed: 7292
processed: 7293
processed: 7294
processed: 7295
processed: 7296
processed: 7297
processed: 7298
processed: 7299
processed: 7300
processed: 7301
processed: 7302
processed: 7303
processed: 7304
processed: 7305
processed: 7306
processed: 7307
processed: 7308
processed: 7309
processed: 7310
processed: 7311
processed: 7312
processe

processed: 7764
processed: 7765
processed: 7766
processed: 7767
processed: 7768
processed: 7769
processed: 7770
processed: 7771
processed: 7772
processed: 7773
processed: 7774
processed: 7775
processed: 7776
processed: 7777
processed: 7778
processed: 7779
processed: 7780
processed: 7781
processed: 7782
processed: 7783
processed: 7784
processed: 7785
processed: 7786
processed: 7787
processed: 7788
processed: 7789
processed: 7790
processed: 7791
processed: 7792
processed: 7793
processed: 7794
processed: 7795
processed: 7796
processed: 7797
processed: 7798
processed: 7799
processed: 7800
processed: 7801
processed: 7802
processed: 7803
processed: 7804
processed: 7805
processed: 7806
processed: 7807
processed: 7808
processed: 7809
processed: 7810
processed: 7811
processed: 7812
processed: 7813
processed: 7814
processed: 7815
processed: 7816
processed: 7817
processed: 7818
processed: 7819
processed: 7820
processed: 7821
processed: 7822
processed: 7823
processed: 7824
processed: 7825
processe

processed: 8277
processed: 8278
processed: 8279
processed: 8280
processed: 8281
processed: 8282
processed: 8283
processed: 8284
processed: 8285
processed: 8286
processed: 8287
processed: 8288
processed: 8289
processed: 8290
processed: 8291
processed: 8292
processed: 8293
processed: 8294
processed: 8295
processed: 8296
processed: 8297
processed: 8298
processed: 8299
processed: 8300
processed: 8301
processed: 8302
processed: 8303
processed: 8304
processed: 8305
processed: 8306
processed: 8307
processed: 8308
processed: 8309
processed: 8310
processed: 8311
processed: 8312
processed: 8313
processed: 8314
processed: 8315
processed: 8316
processed: 8317
processed: 8318
processed: 8319
processed: 8320
processed: 8321
processed: 8322
processed: 8323
processed: 8324
processed: 8325
processed: 8326
processed: 8327
processed: 8328
processed: 8329
processed: 8330
processed: 8331
processed: 8332
processed: 8333
processed: 8334
processed: 8335
processed: 8336
processed: 8337
processed: 8338
processe

processed: 8790
processed: 8791
processed: 8792
processed: 8793
processed: 8794
processed: 8795
processed: 8796
processed: 8797
processed: 8798
processed: 8799
processed: 8800
processed: 8801
processed: 8802
processed: 8803
processed: 8804
processed: 8805
processed: 8806
processed: 8807
processed: 8808
processed: 8809
processed: 8810
processed: 8811
processed: 8812
processed: 8813
processed: 8814
processed: 8815
processed: 8816
processed: 8817
processed: 8818
processed: 8819
processed: 8820
processed: 8821
processed: 8822
processed: 8823
processed: 8824
processed: 8825
processed: 8826
processed: 8827
processed: 8828
processed: 8829
processed: 8830
processed: 8831
processed: 8832
processed: 8833
processed: 8834
processed: 8835
processed: 8836
processed: 8837
processed: 8838
processed: 8839
processed: 8840
processed: 8841
processed: 8842
processed: 8843
processed: 8844
processed: 8845
processed: 8846
processed: 8847
processed: 8848
processed: 8849
processed: 8850
processed: 8851
processe

processed: 9303
processed: 9304
processed: 9305
processed: 9306
processed: 9307
processed: 9308
processed: 9309
processed: 9310
processed: 9311
processed: 9312
processed: 9313
processed: 9314
processed: 9315
processed: 9316
processed: 9317
processed: 9318
processed: 9319
processed: 9320
processed: 9321
processed: 9322
processed: 9323
processed: 9324
processed: 9325
processed: 9326
processed: 9327
processed: 9328
processed: 9329
processed: 9330
processed: 9331
processed: 9332
processed: 9333
processed: 9334
processed: 9335
processed: 9336
Relevance mairix ML:
[[0.32828726 0.11020994 0.09833747 ... 0.11225859 0.12612782 0.17381722]
 [0.26625329 0.11922378 0.14063668 ... 0.08690245 0.13064453 0.09997834]
 [0.16902054 0.14047283 0.10925467 ... 0.10225866 0.31547872 0.14188194]
 ...
 [0.25182263 0.16875737 0.11844617 ... 0.18100925 0.11839404 0.24763754]
 [0.245676   0.11871946 0.13575354 ... 0.08939633 0.13384798 0.37322013]
 [0.20070278 0.24232773 0.19387319 ... 0.09176719 0.10024051 0.20

Obtain fuzzy clusters

In [18]:
import numpy as np
from pyfaddis.lapin import lapin
from pyfaddis.faddis import faddis

from operator import itemgetter

In [19]:
relevance_matrix_LG = np.loadtxt("matrixes/relevmtx_LG")
print(relevance_matrix_LG.shape)
tc = relevance_matrix_LG.T.dot(relevance_matrix_LG)
print(tc.shape)

tc_transformed = lapin(tc)
B, member, contrib, intensity, lat, tt = faddis(tc_transformed)
np.savetxt("clusters/clusters_LG.dat", member)
np.save("clusters/LG_contrib", contrib)
np.save("clusters/LG_intensity", intensity)
np.save("clusters/LG_lat", lat)
np.save("clusters/LG_tt", tt)

(42929, 379)
(379, 379)
Cluster contribution is too small


FileNotFoundError: [Errno 2] No such file or directory: 'cluster/LG_intensity.npy'

In [21]:
relevance_matrix_AI = np.loadtxt("matrixes/relevmtx_AI")
print(relevance_matrix_AI.shape)
tc = relevance_matrix_AI.T.dot(relevance_matrix_AI)
print(tc.shape)

tc_transformed = lapin(tc)
B, member, contrib, intensity, lat, tt = faddis(tc_transformed)
np.savetxt("clusters/clusters_AI.dat", member)
np.save("clusters/AI_contrib", contrib)
np.save("clusters/AI_intensity", intensity)
np.save("clusters/AI_lat", lat)
np.save("clusters/AI_tt", tt)

(20841, 379)
(379, 379)
Cluster contribution is too small


In [22]:
relevance_matrix_ML = np.loadtxt("matrixes/relevmtx_ML")
print(relevance_matrix_ML.shape)
tc = relevance_matrix_ML.T.dot(relevance_matrix_ML)
print(tc.shape)

tc_transformed = lapin(tc)
B, member, contrib, intensity, lat, tt = faddis(tc_transformed)
np.savetxt("clusters/clusters_ML.dat", member)
np.save("clusters/ML_contrib", contrib)
np.save("clusters/ML_intensity", intensity)
np.save("clusters/ML_lat", lat)
np.save("clusters/ML_tt", tt)

(9336, 379)
(379, 379)
Cluster contribution is too small


# Analysis

In [110]:
print(f'ML articles: {len(df_LG)}')
print(f'AI articles: {len(df_AI)}')
print(f'ML (statistics) articles: {len(df_ML)}')

ML articles: 42929
AI articles: 20841
ML (statistics) articles: 9336


In [118]:
member_LG = np.loadtxt('clusters/clusters_LG.dat')
member_AI = np.loadtxt('clusters/clusters_AI.dat')
member_ML = np.loadtxt('clusters/clusters_ML.dat')

In [120]:
terms_LG = []
for i, el in enumerate(strings):
    terms_LG.append((el, member_LG[i, 0], member_LG[i, 1], member_LG[i, 2], member_LG[i, 3], member_LG[i, 4]))

terms_AI = []
for i, el in enumerate(strings):
    terms_AI.append((el, member_AI[i, 0], member_AI[i, 1], member_AI[i, 2], member_AI[i, 3], member_AI[i, 4]))
    
terms_ML = []
for i, el in enumerate(strings):
    terms_ML.append((el, member_ML[i, 0], member_ML[i, 1], member_ML[i, 2]))

#### Machine Leaning (Computer Science)

In [121]:
contrib_LG = np.load('clusters/LG_contrib.npy')
print(f'Cluster contributions: {contrib_LG}')



Cluster contributions: [0.01593554 0.01557557 0.00655408 0.00712074 0.00241163]


In [122]:
print('Machine Learning, Cluster 1:')
terms_LG.sort(reverse=True, key=lambda x: x[1])
for el in terms_LG:
    if el[1] == 0:
        break
    print(f'{el[0]}:\t\t{el[1]}')


Machine Learning, Cluster 1:
data provenance:		0.4337646436749266
2d pca:		0.4200553802871822
bayesian network models:		0.37343141255300594
adversarial learning:		0.33608056630589883
cluster analysis:		0.29552641681032543
markov networks:		0.2428307758986762
distributed database recovery:		0.2425845115367029
data exchange:		0.1974674127407011
data cleaning:		0.19617240053455479
information extraction:		0.1337805882877761
sequential decision making:		0.13071013153836322
anomaly detection:		0.11189830511379048
markov decision processes:		0.10469428507884355
recommender systems:		0.10017294688175708
inverse reinforcement learning:		0.08648519759893851
data locking:		0.08620640658775858
learning to rank:		0.08417138190948344
multi-agent reinforcement learning:		0.06871282152177126
apprenticeship learning:		0.059628387249230516
support vector machines:		0.038684693115777054
boosting:		0.00789701305904788
gaussian processes:		0.006527853448585735
fuzzy representation:		3.200534048444169e-10


In [123]:
print('Machine Learning, Cluster 2:')
terms_LG.sort(reverse=True, key=lambda x: x[2])
for el in terms_LG:
    if el[2] == 0:
        break
    print(f'{el[0]}:\t\t{el[2]}')

Machine Learning, Cluster 2:
data provenance:		0.4396953457030884
2d pca:		0.4261918057130437
adversarial learning:		0.33991644481846445
markov network models:		0.337185005712885
cluster analysis:		0.29965509845437854
bayesian networks:		0.2838477447980121
data cleaning:		0.20021050762244857
data exchange:		0.19748113508204618
distributed data locking:		0.18739884707557178
database recovery:		0.1427764126134109
information extraction:		0.1355731243340465
sequential decision making:		0.1273557894597782
anomaly detection:		0.10996522792090706
markov decision processes:		0.10240506566044434
recommender systems:		0.0993942122933422
inverse reinforcement learning:		0.08781355067614915
learning to rank:		0.07964307184603972
multi-agent reinforcement learning:		0.06636223012671934
apprenticeship learning:		0.06031408968942941
support vector machines:		0.04020082421555917
boosting:		0.007277178418447111
gaussian processes:		0.004412184746553468


In [124]:
print('Machine Learning, Cluster 3:')
terms_LG.sort(reverse=True, key=lambda x: x[3])
for el in terms_LG:
    if el[3] == 0:
        break
    print(f'{el[0]}:\t\t{el[3]}')

Machine Learning, Cluster 3:
learning to rank:		0.415242407867412
sequential decision making:		0.394426726457116
support vector machines:		0.3606143248419246
markov decision processes:		0.3142283090006086
data cleaning:		0.2785466464726828
information extraction:		0.24724563515801362
anomaly detection:		0.22278892218598814
inverse reinforcement learning:		0.22127732072230655
apprenticeship learning:		0.2127356688332793
multi-agent reinforcement learning:		0.18151191850018808
data exchange:		0.16828878348945764
cluster analysis:		0.15635097039667567
recommender systems:		0.13433996479355195
distributed data locking:		0.09872628766826216
2d pca:		0.09811096193279259
data provenance:		0.0966267289547563
gaussian processes:		0.08875219552958068
boosting:		0.08825170205309776
database recovery:		0.07544625232640033
markov network models:		0.06283010452172838
bayesian networks:		0.052930468198218075
adversarial learning:		0.008300701757279474


In [125]:
print('Machine Learning, Cluster 4:')
terms_LG.sort(reverse=True, key=lambda x: x[4])
for el in terms_LG:
    if el[4] == 0:
        break
    print(f'{el[0]}:\t\t{el[4]}')

Machine Learning, Cluster 4:
learning to rank:		0.428250270584852
sequential decision making:		0.40188377577237333
support vector machines:		0.37609595455774997
markov decision processes:		0.3202858950697762
data cleaning:		0.2726319991639786
information extraction:		0.24559003539328272
inverse reinforcement learning:		0.2236187067978763
anomaly detection:		0.2216679317871739
apprenticeship learning:		0.2170748235851293
multi-agent reinforcement learning:		0.182571231095051
data exchange:		0.15565322824366057
cluster analysis:		0.13328470956239333
recommender systems:		0.1302826306169525
distributed database recovery:		0.1081334582627704
gaussian processes:		0.09153375397976467
boosting:		0.09099344956053369
2d pca:		0.059541528210488706
data provenance:		0.055173096029132465
data locking:		0.037500644647982466
bayesian network models:		0.03283750788195399
markov networks:		0.020742726760064008


In [126]:
print('Machine Learning, Cluster 5:')
terms_LG.sort(reverse=True, key=lambda x: x[5])
for el in terms_LG:
    if el[5] == 0:
        break
    print(f'{el[0]}:\t\t{el[5]}')

Machine Learning, Cluster 5:
query representation:		0.9263917911984688
fuzzy clustering:		0.22333130825872058
mixture modeling:		0.19808927760468295
hierarchical data models:		0.1403275886325566
inconsistent data:		0.10157868558958608
document topic models:		0.08242752357094182
test collections:		0.05483188201885137
probabilistic retrieval models:		0.05083477963391369
temporal reasoning:		0.04414367642360795
vagueness and fuzzy logic:		0.030620996307408
neuro-fuzzy approach:		0.029732505973361467
kernel independent components:		0.02704075883930022
search results deduplication:		0.02613326096397946
shape representations:		0.025940410527286246
database query processing and optimization:		0.019880117068685553
active learning:		0.019522033866878466
sparse tensor:		0.019032182537057133
canonical correlation analysis:		0.01846109330491038
online learning settings:		0.015424708539636922
object recognition:		0.012936945940312033
models of learning:		0.01269886074416544
structured query languag

#### AI

In [117]:
contrib_AI = np.load('clusters/AI_contrib.npy')
print(f'Cluster contributions: {contrib_AI}')


Cluster contributions: [0.02048352 0.01977237 0.0055853  0.00547745 0.00286389]


In [104]:
print('AI, Cluster 1:')
terms_AI.sort(reverse=True, key=lambda x: x[1])
for el in terms_AI:
    if el[1] == 0:
        break
    print(f'{el[0]}:\t\t{el[1]}')

AI, Cluster 1:
support vector machines:		0.3819018050052615
anomaly detection:		0.35480000486855523
distributed database recovery:		0.3428976977378799
2d pca:		0.326197441647439
gaussian processes:		0.27492639083327725
boosting:		0.26870030745010426
markov decision processes:		0.25729433468901614
cluster analysis:		0.2526371877594142
bayesian network models:		0.24137518484187223
apprenticeship learning:		0.17163801632548897
markov networks:		0.1565495007637852
recommender systems:		0.14674827846309288
information extraction:		0.13031835919783943
data locking:		0.12476473857591884
learning to rank:		0.12182756680249088
data cleaning:		0.11299497117537075
adversarial learning:		0.09461650350790746
sequential decision making:		0.0917504874887954
data exchange:		0.09019242142565233
multi-agent reinforcement learning:		0.05835104483678031
data provenance:		0.05028247248097171
inverse reinforcement learning:		0.03335358561056711
inconsistent data:		2.428009045602563e-10
probabilistic retriev

In [105]:
print('AI, Cluster 2:')
terms_AI.sort(reverse=True, key=lambda x: x[2])
for el in terms_AI:
    if el[2] == 0:
        break
    print(f'{el[0]}:\t\t{el[2]}')

AI, Cluster 2:
support vector machines:		0.4097246523865225
2d pca:		0.34313786294874077
anomaly detection:		0.3420705733991426
gaussian processes:		0.27984167662897663
distributed data locking:		0.2755050924666692
boosting:		0.26740165675834565
markov decision processes:		0.25369554511481973
cluster analysis:		0.236124752887918
markov network models:		0.2244326824453322
database recovery:		0.21509584815670188
bayesian networks:		0.1851059051914503
apprenticeship learning:		0.15936089161891304
recommender systems:		0.14526914399850377
information extraction:		0.12886503095103707
learning to rank:		0.11825584537426756
data cleaning:		0.1143902958207284
adversarial learning:		0.09121618218204301
data exchange:		0.08954881371387764
sequential decision making:		0.08139716720414208
data provenance:		0.06151437796089847
multi-agent reinforcement learning:		0.04813867495370546
inverse reinforcement learning:		0.03733584259163887


In [106]:
print('AI, Cluster 3:')
terms_AI.sort(reverse=True, key=lambda x: x[3])
for el in terms_AI:
    if el[3] == 0:
        break
    print(f'{el[0]}:\t\t{el[3]}')

AI, Cluster 3:
support vector machines:		0.467073855287433
cluster analysis:		0.42633169593482256
anomaly detection:		0.38263054438290045
apprenticeship learning:		0.3035107462080092
data provenance:		0.25017700624479583
2d pca:		0.24893826470473518
distributed database recovery:		0.2389726853396938
sequential decision making:		0.2305779247640998
multi-agent reinforcement learning:		0.2050782724160649
markov decision processes:		0.16010643555240034
inverse reinforcement learning:		0.10375659113238551
boosting:		0.10301467661479788
learning to rank:		0.10221596415369721
data locking:		0.08693069392538968
bayesian network models:		0.0846975436514543
adversarial learning:		0.06514882724722768
markov networks:		0.054920410755000225
recommender systems:		0.053040476032016746
information extraction:		0.029322720497506245
gaussian processes:		0.02858323065359169
data exchange:		0.02563977377424034
data cleaning:		0.008366298954496835


In [128]:
print('AI, Cluster 4:')
terms_AI.sort(reverse=True, key=lambda x: x[4])
for el in terms_AI:
    if el[4] == 0:
        break
    print(f'{el[0]}:\t\t{el[4]}')

AI, Cluster 4:
support vector machines:		0.4684796728498491
cluster analysis:		0.44121959117922027
anomaly detection:		0.3802025495308368
apprenticeship learning:		0.31479379045164757
data provenance:		0.2693193600689445
sequential decision making:		0.24376337047301355
2d pca:		0.2346307776315456
multi-agent reinforcement learning:		0.21953744136277295
distributed data locking:		0.16965400300612637
markov decision processes:		0.14557678366360427
database recovery:		0.13255615429656314
inverse reinforcement learning:		0.1098390911904022
learning to rank:		0.09785642703149249
boosting:		0.08077603625554645
adversarial learning:		0.06002314167790333
markov network models:		0.057330509891136874
bayesian networks:		0.04725557762851908
recommender systems:		0.040380823879650356
data exchange:		0.016948052271871076
information extraction:		0.015785433031568433


In [127]:
print('AI, Cluster 5:')
terms_AI.sort(reverse=True, key=lambda x: x[5])
for el in terms_AI:
    if el[5] == 0:
        break
    print(f'{el[0]}:\t\t{el[5]}')

AI, Cluster 5:
learning to rank:		0.44415922476065994
information extraction:		0.4226382878998822
markov decision processes:		0.3395194849457521
data cleaning:		0.3063693800407274
data provenance:		0.267561113900364
recommender systems:		0.2583785315554916
boosting:		0.2485458993362883
2d pca:		0.23175675960452188
sequential decision making:		0.18900259067647765
adversarial learning:		0.18560273865739266
data exchange:		0.1601073546022096
apprenticeship learning:		0.12527786273345767
support vector machines:		0.1216555585320845
anomaly detection:		0.10568865269111494
cluster analysis:		0.09891908835740307
inverse reinforcement learning:		0.07839098120327641
gaussian processes:		0.05031403288789675
multi-agent reinforcement learning:		0.0436258966021172
distributed data locking:		0.043213988352151414
database recovery:		0.03379741917409715
markov network models:		0.0130211208083743
bayesian networks:		0.010749948130869429


#### Machine Learning (Statistics)

In [116]:

contrib_ML = np.load('clusters/ML_contrib.npy')
print(f'Cluster contributions: {contrib_ML}')

Cluster contributions: [0.01943845 0.01867466 0.00483639]


In [107]:
print('Machine Learning (Statistics), Cluster 1:')
terms_ML.sort(reverse=True, key=lambda x: x[1])
for el in terms_ML:
    if el[1] == 0:
        break
    print(f'{el[0]}:\t\t{el[1]}')

Machine Learning (Statistics), Cluster 1:
adversarial learning:		0.3736923549301199
learning to rank:		0.3685328483004863
distributed database recovery:		0.3527455968860433
boosting:		0.3469433442790289
multi-agent reinforcement learning:		0.3064425099811277
bayesian network models:		0.3027894284894846
support vector machines:		0.28473314716948805
data cleaning:		0.1949433135574657
markov networks:		0.19021091936548537
data exchange:		0.14816048088094896
sequential decision making:		0.13707836249628666
recommender systems:		0.13492380986079155
markov decision processes:		0.13441304175391872
apprenticeship learning:		0.1276275507342564
data locking:		0.11943487621544069
gaussian processes:		0.11792717160453721
data provenance:		0.09803808666851291
cluster analysis:		0.07524444295746532
information extraction:		0.03250740709333087
inverse reinforcement learning:		0.02919529880692395
anomaly detection:		0.015234436604552813
2d pca:		0.002436450163272978
fuzzy representation:		5.3740788134

In [108]:
print('Machine Learning (Statistics), Cluster 2:')
terms_ML.sort(reverse=True, key=lambda x: x[2])
for el in terms_ML:
    if el[2] == 0:
        break
    print(f'{el[0]}:\t\t{el[2]}')

Machine Learning (Statistics), Cluster 2:
adversarial learning:		0.37946689111894366
learning to rank:		0.37509053164650547
boosting:		0.3526648123941726
multi-agent reinforcement learning:		0.31009805055848577
support vector machines:		0.2899971081974449
distributed data locking:		0.27854805037219893
markov network models:		0.26706665540265795
bayesian networks:		0.23314226913140526
database recovery:		0.20352167259633477
data cleaning:		0.1976597840373411
data exchange:		0.14995875920557247
sequential decision making:		0.1388511217453459
recommender systems:		0.13612519935719503
markov decision processes:		0.13590285676820169
apprenticeship learning:		0.12878592908564462
gaussian processes:		0.11897898622660151
data provenance:		0.09841994630344361
cluster analysis:		0.0758032826760344
information extraction:		0.03104780596555398
inverse reinforcement learning:		0.028446852603110584
anomaly detection:		0.01431916015369153
2d pca:		0.0011823843825989517


In [109]:
print('Machine Learning (Statistics), Cluster 3:')
terms_ML.sort(reverse=True, key=lambda x: x[3])
for el in terms_ML:
    if el[3] == 0:
        break
    print(f'{el[0]}:\t\t{el[3]}')

Machine Learning (Statistics), Cluster 3:
data provenance:		0.49455791368616586
apprenticeship learning:		0.3928307404541719
sequential decision making:		0.33246769677579496
bayesian network models:		0.2827993911906054
multi-agent reinforcement learning:		0.24760203499107813
gaussian processes:		0.24455092106236578
adversarial learning:		0.2124162795962565
inverse reinforcement learning:		0.20807031598371856
cluster analysis:		0.19438661526075787
markov networks:		0.17765378251039915
data cleaning:		0.17685954526258704
2d pca:		0.1701004182528179
markov decision processes:		0.1511645008204334
information extraction:		0.13423831610767478
support vector machines:		0.0887637617092758
boosting:		0.08716866304026843
distributed data locking:		0.06502219174845411
learning to rank:		0.06404264874232415
anomaly detection:		0.05481575333800362
database recovery:		0.04760784367223929
data exchange:		0.03731493000949169
recommender systems:		0.012603563454247952
information visualization:		2.2150

## The following is a test ----

## ----